Movie Recommendation engine
1. your recommended engine will return maximum 10 movies name if an user search for a particular movie.
Evaluation

2. Recommended engine must return 5 movie names and maximum it can return 10 movie names if an user search for a particular movie. This recommender engine should not give suggestion in between 1 to 4 and 6 to 10 it have to return 5 movie names or 10 movie names.

Analysis

1. This will be a content-based filtering which make recomemndation based on the attributes

2. As a netflix binge watcher, the category are very well separated by the type, so we will want recommendation for TV shows if we are already watching one,vis-a-versa for movies

In [40]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
data = pd.read_csv(r"C:\Users\Yong Ren\Documents\Jupyter\netflix_titles.csv")
data.head(15)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,"June 1, 2020",2019,TV-MA,95 min,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."
7,s8,Movie,187,Kevin Reynolds,"Samuel L. Jackson, John Heard, Kelly Rowan, Cl...",United States,"November 1, 2019",1997,R,119 min,Dramas,After one of his high school students attacks ...
8,s9,Movie,706,Shravan Kumar,"Divya Dutta, Atul Kulkarni, Mohan Agashe, Anup...",India,"April 1, 2019",2019,TV-14,118 min,"Horror Movies, International Movies","When a doctor goes missing, his psychiatrist w..."
9,s10,Movie,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",India,"December 15, 2017",2008,TV-MA,143 min,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...


In [42]:
# Understanding the data
data.shape

(7787, 12)

In [43]:
data.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

In [44]:
# Drop missing data in the excel, it is a personal choice not to drop country cause tbh I will not look at the country for recommendation
data = data.dropna(subset=['director','cast', 'rating'])
data.shape

(4975, 12)

In [45]:
data = data.drop(['show_id','date_added','release_year','duration','description'], axis = 1)
data.head()

,type,title,director,cast,country,rating,listed_in
1,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,TV-MA,"Dramas, International Movies"
2,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,R,"Horror Movies, International Movies"
3,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,PG-13,"Action & Adventure, Independent Movies, Sci-Fi..."
4,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,PG-13,Dramas
5,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"


In [46]:
data.isnull().sum()

type           0
title          0
director       0
cast           0
country      167
rating         0
listed_in      0
dtype: int64

In [47]:
data.groupby('type').count()

,title,director,cast,country,rating,listed_in
type,,,,,,
Movie,4830,4830,4830,4673,4830,4830
TV Show,145,145,145,135,145,145


In [48]:
# splitting the data, perfect place to use the Regular expression
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['cast'] = [re.sub(r'[^\w]', '', t) for t in data['cast']]

data['listed_in'] = [t.lower() for t in data['listed_in']]
data['cast'] = [t.lower() for t in data['cast']]

# creating bag of words
data["combined"] = data['listed_in'] + '  ' + data['title'] + ' ' + data['cast'] + ' ' + data['country'] 
data.drop(["cast", "listed_in", "country", 'director'],axis=1,inplace=True)
data.head()

,type,title,rating,combined
1,Movie,7:19,TV-MA,dramas international movies 7:19 demián bichi...
2,Movie,23:59,R,horror movies international movies 23:59 tedd...
3,Movie,9,PG-13,action adventure independent movies scifi fa...
4,Movie,21,PG-13,dramas 21 jim sturgess kevin spacey kate bosw...
5,TV Show,46,TV-MA,international tv shows tv dramas tv mysteries ...


In [49]:
#separate tv shows and movies
Tv = data[data['type'] =='TV Show'].reset_index()
Mov = data[data['type'] =='Movie'].reset_index()
Tv.head()

,index,type,title,rating,combined
0,5,TV Show,46,TV-MA,international tv shows tv dramas tv mysteries ...
1,132,TV Show,7 (Seven),TV-14,tv shows 7 (Seven) rahman havish regina cassa...
2,218,TV Show,A Queen Is Born,TV-14,international tv shows reality tv A Queen Is ...
3,260,TV Show,A.I.C.O.,TV-14,anime series international tv shows A.I.C.O. ...
4,276,TV Show,Abby Hatcher,TV-Y,kids tv Abby Hatcher macy drouin wyatt white ...


In [50]:
# main

vectorizer = CountVectorizer()
recommended_movies = []

# matrixTv = vectorizer.fit_transform(Tv['combined'].values.astype('U'))
# idx = Tv[Tv['title'] == search].index.item()
# similar = cosine_similarity(matrix,matrix)
# score = pd.Series(similar[idx]).sort_values(ascending = False)
# top_10 = list(score.iloc[1:11].index)

# for i in top_10:
#     recommended_movies.append((Tv['title'])[i])
# recommended_movies

def netflix_recommender(search):
    if search in Tv['title'].values:
        matrixTv = vectorizer.fit_transform(Tv['combined'].values.astype('U'))
        #find the index of the item been searched
        idx = Tv[Tv['title'] == search].index.item()
        similar = cosine_similarity(matrixTv,matrixTv)
        score = pd.Series(similar[idx]).sort_values(ascending = False)
        top_10 = list(score.iloc[1:11].index)
        
        for i in top_10:
            recommended_movies.append((Tv['title'])[i])
        return recommended_movies
    
    elif search in Mov['title'].values:
        matrixMov = vectorizer.fit_transform(Mov['combined'].values.astype('U'))
        idx = Mov[Mov['title'] == search].index.item()
        similar = cosine_similarity(matrixMov,matrixmov)
        score = pd.Series(similar[idx]).sort_values(ascending = False)
        top_10 = list(score.iloc[1:11].index)
        
        for i in top_10:
            recommended_movies.append((Tv['title'])[i])
            
        return recommended_movies
    
    else :
        print("Error! No such title found")
        

In [51]:
netflix_recommender('Almost Happy')

['Valeria',
 'Fary : Hexagone',
 'Mr. Sunshine',
 'Gonul',
 'Jack Taylor',
 'GHOUL',
 'Sacred Games',
 'Camelia la Texana',
 'Reply 1994',
 'Before 30']